In [22]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, export_graphviz
# from IPython.display import Image
import graphviz
from sklearn.preprocessing import Imputer
from io import StringIO

In [23]:
df_test = pd.read_csv('internetads_test.csv', encoding='gbk', na_values=['?'])
df_train = pd.read_csv('internetads_train.csv', encoding='gbk', na_values=['?'])
predictors = list(df_test)
respond_val = predictors.pop()

X_train = df_train.loc[:, predictors]
y_train = df_train[respond_val]
X_test = df_test.loc[:, predictors]
y_test = df_test[respond_val]

In [24]:
def isInt(series):
    for entry in series:
        try:
            _ = int(entry)
        except ValueError:
            if not entry.strip() == '?':
                return False
    return True


def clean_type_by_col(df):
    nan = float('NaN')
    for col in predictors:
        if df[col].dtypes == 'object':
            if isInt(df[col]):
                df[col] = [int(d) if d.strip() != '?' else nan for d in df[col]]
            else:
                df[col] = [float(d) if d.strip() != '?' else nan for d in df[col]]
        # else:
        #     df[col] = [d if d.strip() != '?' else nan for d in df[col]]

clean_type_by_col(X_train)
clean_type_by_col(X_test)

In [25]:
imp = Imputer(strategy='most_frequent', axis=0)  # by column
X_train = imp.fit_transform(X_train)
X_test = imp.fit_transform(X_test)

In [29]:
def plot_tree(clf, plot_name):
    dot_data = StringIO()
    export_graphviz(clf, out_file=dot_data, feature_names=predictors, 
                    class_names=['ad.', 'nonad.'], filled=True, 
                    rounded=True, special_characters=True)
    graphviz.Source(dot_data.getvalue()).render(plot_name, view=False)

for max_depth in (3, 5):
    tree = DecisionTreeClassifier(max_depth=max_depth)
    tree.fit(X_train, y_train)
    plot_tree(tree, 'depth_{:d}_tree'.format(max_depth))
    error_rate = sum(tree.predict(X_test) != y_test) / len(y_test)
    print('Tree max_depth={:d}, error rate={:.3f}'.format(max_depth, error_rate))

Tree max_depth=3, error rate=0.060
Tree max_depth=5, error rate=0.054
